In [1]:
from keras.datasets import mnist, cifar10
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np

In [62]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

In [3]:
def build_generator():

    noise_shape = (100,) #1D array of size 100 (latent vector / noise)



    model = Sequential()

    model.add(Dense(256, input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))

    model.summary()

    noise = Input(shape=noise_shape)
    img = model(noise)    

    return Model(noise, img)

In [4]:
def build_discriminator():


    model = Sequential()

    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

In [65]:
def train(epochs, batch_size=128, save_interval=500):
    (x_train,_),(_,_)=mnist.load_data()
    
    x_train=(x_train.astype(np.float32)-127.5)/ 127.5
    
    X_train= np.expand_dims(x_train, axis=3)
    
    valid=np.ones((batch_size, 1))
    fake= np.zeros((batch_size, 1))
    
    
    for epoch in range(epochs):
        idx= np.random.randint(0, X_train.shape[0], batch_size)
        imgs= X_train[idx]
        
        ###Train Discriminator
        noise= np.random.normal(0, 1, (batch_size, 100))
        gen_imgs= generator.predict(noise)
        
        d_loss_real= discriminator.train_on_batch(imgs, valid)
        d_loss_fake= discriminator.train_on_batch(gen_imgs, fake)
        d_loss= 0.5 * np.add(d_loss_real, d_loss_fake)
        
        
        ####Train Generator
        noise= np.random.normal(0, 1, (batch_size, 100))
        g_loss= combined.train_on_batch(noise, valid)
        
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

        if epoch % save_interval == 0:
            save_imgs(epoch)

def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("generated_images_mnist/mnist_%d.png" % epoch)
    plt.close()


In [6]:
optimizer = Adam(0.0002, 0.5)

In [7]:
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 dense (Dense)               (None, 512)               1573376   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 257       
                                                                 
Total params: 1,704,961
Trainable params: 1,704,961
Non-

In [8]:
generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 256)               25856     
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 256)               0         
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense_4 (Dense)             (None, 512)               131584    
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 512)               0         
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2048      
 hNormalization)                                      

In [9]:
z = Input(shape=(100,))   
img = generator(z)

In [10]:
discriminator.trainable=False

In [11]:
valid= discriminator(img)

In [12]:
combined=Model(z, valid)
combined.compile(loss="binary_crossentropy", optimizer=optimizer)

In [15]:
train(epochs=45000, batch_size=128, save_interval=1000)

4/4 [==============================] - 0s 5ms/step
0 [D loss: 0.512509, acc.: 82.81%] [G loss: 0.476387]
4/4 [==============================] - 0s 5ms/step
1 [D loss: 0.454414, acc.: 75.00%] [G loss: 0.373785]
4/4 [==============================] - 0s 0s/step
2 [D loss: 0.467275, acc.: 56.64%] [G loss: 0.321318]
4/4 [==============================] - 0s 0s/step
3 [D loss: 0.502590, acc.: 55.08%] [G loss: 0.293150]
4/4 [==============================] - 0s 0s/step
4 [D loss: 0.488862, acc.: 60.55%] [G loss: 0.283080]
4/4 [==============================] - 0s 0s/step
5 [D loss: 0.465587, acc.: 64.84%] [G loss: 0.313891]
4/4 [==============================] - 0s 0s/step
6 [D loss: 0.409235, acc.: 78.91%] [G loss: 0.357929]
4/4 [==============================] - 0s 5ms/step
7 [D loss: 0.396030, acc.: 80.08%] [G loss: 0.413431]
4/4 [==============================] - 0s 0s/step
8 [D loss: 0.439876, acc.: 78.52%] [G loss: 0.445798]
4/4 [==============================] - 0s 0s/step
9 [D loss: 

4/4 [==============================] - 0s 0s/step
77 [D loss: 0.678059, acc.: 63.28%] [G loss: 1.688377]
4/4 [==============================] - 0s 5ms/step
78 [D loss: 0.710932, acc.: 57.03%] [G loss: 1.571760]
4/4 [==============================] - 0s 0s/step
79 [D loss: 0.706837, acc.: 62.50%] [G loss: 1.695444]
4/4 [==============================] - 0s 0s/step
80 [D loss: 0.735602, acc.: 54.30%] [G loss: 1.724092]
4/4 [==============================] - 0s 1ms/step
81 [D loss: 0.697464, acc.: 56.64%] [G loss: 1.585958]
4/4 [==============================] - 0s 0s/step
82 [D loss: 0.693622, acc.: 64.06%] [G loss: 1.585880]
4/4 [==============================] - 0s 0s/step
83 [D loss: 0.772195, acc.: 54.30%] [G loss: 1.493972]
4/4 [==============================] - 0s 0s/step
84 [D loss: 0.643334, acc.: 67.19%] [G loss: 1.709166]
4/4 [==============================] - 0s 5ms/step
85 [D loss: 0.660998, acc.: 64.06%] [G loss: 1.615292]
4/4 [==============================] - 0s 0s/step
86

4/4 [==============================] - 0s 0s/step
154 [D loss: 0.718740, acc.: 58.20%] [G loss: 1.500707]
4/4 [==============================] - 0s 5ms/step
155 [D loss: 0.661782, acc.: 61.33%] [G loss: 1.454556]
4/4 [==============================] - 0s 0s/step
156 [D loss: 0.730652, acc.: 60.94%] [G loss: 1.396356]
4/4 [==============================] - 0s 5ms/step
157 [D loss: 0.680955, acc.: 58.98%] [G loss: 1.343059]
4/4 [==============================] - 0s 0s/step
158 [D loss: 0.684523, acc.: 62.50%] [G loss: 1.425826]
4/4 [==============================] - 0s 0s/step
159 [D loss: 0.714300, acc.: 55.47%] [G loss: 1.389331]
4/4 [==============================] - 0s 0s/step
160 [D loss: 0.675600, acc.: 60.55%] [G loss: 1.365131]
4/4 [==============================] - 0s 0s/step
161 [D loss: 0.691433, acc.: 61.33%] [G loss: 1.403165]
4/4 [==============================] - 0s 5ms/step
162 [D loss: 0.683161, acc.: 59.38%] [G loss: 1.427913]
4/4 [==============================] - 0s 0

4/4 [==============================] - 0s 1ms/step
231 [D loss: 0.767603, acc.: 47.27%] [G loss: 1.436258]
4/4 [==============================] - 0s 1ms/step
232 [D loss: 0.680283, acc.: 63.67%] [G loss: 1.436350]
4/4 [==============================] - 0s 345us/step
233 [D loss: 0.669249, acc.: 58.20%] [G loss: 1.396248]
4/4 [==============================] - 0s 668us/step
234 [D loss: 0.714667, acc.: 62.89%] [G loss: 1.365329]
4/4 [==============================] - 0s 1ms/step
235 [D loss: 0.719924, acc.: 53.52%] [G loss: 1.399462]
4/4 [==============================] - 0s 1ms/step
236 [D loss: 0.668668, acc.: 60.94%] [G loss: 1.388700]
4/4 [==============================] - 0s 1ms/step
237 [D loss: 0.709617, acc.: 59.38%] [G loss: 1.433035]
4/4 [==============================] - 0s 1ms/step
238 [D loss: 0.728974, acc.: 52.34%] [G loss: 1.418402]
4/4 [==============================] - 0s 1ms/step
239 [D loss: 0.702154, acc.: 61.33%] [G loss: 1.393523]
4/4 [============================

307 [D loss: 0.654240, acc.: 62.89%] [G loss: 1.328594]
4/4 [==============================] - 0s 2ms/step
308 [D loss: 0.645005, acc.: 62.11%] [G loss: 1.293883]
4/4 [==============================] - 0s 2ms/step
309 [D loss: 0.657392, acc.: 60.55%] [G loss: 1.375111]
4/4 [==============================] - 0s 3ms/step
310 [D loss: 0.630779, acc.: 66.80%] [G loss: 1.294646]
4/4 [==============================] - 0s 2ms/step
311 [D loss: 0.703698, acc.: 57.42%] [G loss: 1.330068]
4/4 [==============================] - 0s 1ms/step
312 [D loss: 0.673524, acc.: 62.89%] [G loss: 1.312832]
4/4 [==============================] - 0s 2ms/step
313 [D loss: 0.650790, acc.: 61.33%] [G loss: 1.232298]
4/4 [==============================] - 0s 2ms/step
314 [D loss: 0.693698, acc.: 54.69%] [G loss: 1.253856]
4/4 [==============================] - 0s 1ms/step
315 [D loss: 0.642939, acc.: 64.45%] [G loss: 1.302656]
4/4 [==============================] - 0s 2ms/step
316 [D loss: 0.623423, acc.: 62.50%] 

...

4/4 [==============================] - 0s 8ms/step
21137 [D loss: 0.275144, acc.: 88.67%] [G loss: 2.551395]
4/4 [==============================] - 0s 2ms/step
21138 [D loss: 0.428136, acc.: 78.91%] [G loss: 2.387628]
4/4 [==============================] - 0s 3ms/step
21139 [D loss: 0.331404, acc.: 85.55%] [G loss: 2.377243]
4/4 [==============================] - 0s 5ms/step
21140 [D loss: 0.434058, acc.: 78.52%] [G loss: 2.575181]
4/4 [==============================] - 0s 3ms/step
21141 [D loss: 0.461379, acc.: 79.69%] [G loss: 2.937562]
4/4 [==============================] - 0s 2ms/step
21142 [D loss: 0.389716, acc.: 82.03%] [G loss: 3.988503]
4/4 [==============================] - 0s 2ms/step
21143 [D loss: 0.520303, acc.: 75.39%] [G loss: 6.608280]
4/4 [==============================] - 0s 3ms/step
21144 [D loss: 0.846269, acc.: 73.05%] [G loss: 9.322859]
4/4 [==============================] - 0s 3ms/step
21145 [D loss: 3.919810, acc.: 42.19%] [G loss: 6.470042]
4/4 [==============

4/4 [==============================] - 0s 32ms/step
21212 [D loss: 0.362917, acc.: 82.81%] [G loss: 2.500002]
4/4 [==============================] - 0s 48ms/step
21213 [D loss: 0.391876, acc.: 80.86%] [G loss: 1.933152]
4/4 [==============================] - 0s 27ms/step
21214 [D loss: 0.342991, acc.: 84.38%] [G loss: 3.071237]
4/4 [==============================] - 0s 60ms/step
21215 [D loss: 0.323173, acc.: 87.11%] [G loss: 2.781431]
4/4 [==============================] - 0s 46ms/step
21216 [D loss: 0.304998, acc.: 87.11%] [G loss: 5.586966]
4/4 [==============================] - 0s 47ms/step
21217 [D loss: 0.296026, acc.: 85.16%] [G loss: 2.898307]
4/4 [==============================] - 0s 31ms/step
21218 [D loss: 0.343413, acc.: 82.81%] [G loss: 4.083247]
4/4 [==============================] - 0s 104ms/step
21219 [D loss: 0.319214, acc.: 85.94%] [G loss: 2.732691]
4/4 [==============================] - 0s 39ms/step
21220 [D loss: 0.285569, acc.: 86.72%] [G loss: 3.513131]
4/4 [====

21286 [D loss: 0.451971, acc.: 79.30%] [G loss: 2.048185]
4/4 [==============================] - 0s 41ms/step
21287 [D loss: 0.389735, acc.: 79.69%] [G loss: 2.128381]
4/4 [==============================] - 0s 16ms/step
21288 [D loss: 0.373370, acc.: 84.77%] [G loss: 2.362043]
4/4 [==============================] - 0s 92ms/step
21289 [D loss: 0.367277, acc.: 84.77%] [G loss: 2.223563]
4/4 [==============================] - 0s 40ms/step
21290 [D loss: 0.295335, acc.: 88.67%] [G loss: 2.743389]
4/4 [==============================] - 0s 45ms/step
21291 [D loss: 0.360610, acc.: 82.03%] [G loss: 2.641723]
4/4 [==============================] - 0s 48ms/step
21292 [D loss: 0.309538, acc.: 84.38%] [G loss: 2.598796]
4/4 [==============================] - 0s 36ms/step
21293 [D loss: 0.341991, acc.: 82.81%] [G loss: 2.645799]
4/4 [==============================] - 0s 16ms/step
21294 [D loss: 0.347407, acc.: 85.55%] [G loss: 2.281323]
4/4 [==============================] - 0s 32ms/step
21295 [D l

4/4 [==============================] - 0s 34ms/step
21361 [D loss: 0.314517, acc.: 85.94%] [G loss: 2.467923]
4/4 [==============================] - 0s 50ms/step
21362 [D loss: 0.432910, acc.: 78.91%] [G loss: 3.053373]
4/4 [==============================] - 0s 49ms/step
21363 [D loss: 0.328903, acc.: 84.38%] [G loss: 4.834951]
4/4 [==============================] - 0s 40ms/step
21364 [D loss: 0.487872, acc.: 76.95%] [G loss: 2.252982]
4/4 [==============================] - 0s 14ms/step
21365 [D loss: 0.315525, acc.: 84.38%] [G loss: 3.054417]
4/4 [==============================] - 0s 27ms/step
21366 [D loss: 0.372036, acc.: 84.77%] [G loss: 3.072062]
4/4 [==============================] - 0s 40ms/step
21367 [D loss: 0.286394, acc.: 88.28%] [G loss: 2.538206]
4/4 [==============================] - 0s 100ms/step
21368 [D loss: 0.387140, acc.: 81.64%] [G loss: 3.454446]
4/4 [==============================] - 0s 45ms/step
21369 [D loss: 0.365169, acc.: 83.20%] [G loss: 3.562390]
4/4 [====

KeyboardInterrupt: 